In [1]:
import pandas as pd
import numpy as np

In [2]:
! ls -lrt

total 338656
-rw-r--r-- 1 rafael rafael 140600832 jun  5  2020  city_temperature.csv
-rw-r--r-- 1 rafael rafael 205946108 nov  4  2020 'NOOA_weather_all_cities 2019.csv'
-rw-r--r-- 1 rafael rafael     25879 dez  2  2020  lisbon-houses.csv
-rw-r--r-- 1 rafael rafael     49681 jan 17  2022  seattle-weather.csv
-rw-r--r-- 1 rafael rafael      9450 nov  1 16:57 'Project - Lisbon House Prices.ipynb'
-rw-r--r-- 1 rafael rafael    132384 nov  1 17:45  Climate_twitter.csv
-rw-r--r-- 1 rafael rafael       337 nov  1 18:29  README.md


In [3]:
df_raw= pd.read_csv('lisbon-houses.csv')

In [4]:
df_raw.dtypes

Id                   int64
Condition           object
PropertyType        object
PropertySubType     object
Bedrooms             int64
Bathrooms            int64
AreaNet              int64
AreaGross            int64
Parking              int64
Latitude           float64
Longitude          float64
Country             object
District            object
Municipality        object
Parish              object
Price M2             int64
Price                int64
dtype: object

In [5]:
df_raw

,Id,Condition,PropertyType,PropertySubType,Bedrooms,Bathrooms,AreaNet,AreaGross,Parking,Latitude,Longitude,Country,District,Municipality,Parish,Price M2,Price
0,101,Used,Homes,Apartment,3,1,76,152,0,38.7792,-9.1186,Portugal,Lisboa,Lisboa,Olivais,2463,198000
1,102,Used,Homes,Duplex,5,3,190,380,0,38.7056,-9.1784,Portugal,Lisboa,Lisboa,Alcantara,3125,1270000
2,103,Used,Homes,Apartment,1,1,26,52,0,38.7058,-9.1639,Portugal,Lisboa,Lisboa,Estrela,4005,140000
3,104,Used,Homes,Apartment,5,4,185,370,0,38.7466,-9.1640,Portugal,Lisboa,Lisboa,Sao Domingos de Benfica,3412,995000
4,105,Used,Homes,Apartment,7,1,150,300,0,38.7323,-9.1287,Portugal,Lisboa,Lisboa,Arroios,3277,570000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,346,For Refurbishment,Homes,Apartment,2,1,66,132,0,38.7460,-9.1769,Portugal,Lisboa,Lisboa,Benfica,2727,196500
242,347,For Refurbishment,Homes,Apartment,3,1,75,150,0,38.7177,-9.1341,Portugal,Lisboa,Lisboa,Santa Maria Maior,4807,245000
243,348,For Refurbishment,Single Habitation,Townhouse Dwelling,2,1,90,180,0,38.7694,-9.1138,Portugal,Lisboa,Lisboa,Olivais,2463,486000
244,349,For Refurbishment,Single Habitation,Dwelling,4,3,121,242,0,38.7030,-9.2044,Portugal,Lisboa,Lisboa,Belem,3542,375000


In [7]:
df_raw.shape

(246, 17)

In [12]:
df_raw.columns

Index(['Id', 'Condition', 'PropertyType', 'PropertySubType', 'Bedrooms',
       'Bathrooms', 'AreaNet', 'AreaGross', 'Parking', 'Latitude', 'Longitude',
       'Country', 'District', 'Municipality', 'Parish', 'Price M2', 'Price'],
      dtype='object')

In [21]:
df_raw.describe(include='all')

,Id,Condition,PropertyType,PropertySubType,Bedrooms,Bathrooms,AreaNet,AreaGross,Parking,Latitude,Longitude,Country,District,Municipality,Parish,Price M2,Price
count,246.000000,246,246,246,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246.000000,246,246,246,246,246.000000,2.460000e+02
unique,NaN,4,2,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,24,NaN,NaN
top,NaN,New,Homes,Apartment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Portugal,Lisboa,Lisboa,Marvila,NaN,NaN
freq,NaN,96,239,226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,246,246,246,51,NaN,NaN
mean,226.670732,NaN,NaN,NaN,2.491870,1.845528,109.662602,219.325203,0.154472,38.733106,-9.140051,NaN,NaN,NaN,NaN,3487.873984,5.516588e+05
std,72.308419,NaN,NaN,NaN,1.354484,1.034369,66.261664,132.523328,0.503579,0.020778,0.032356,NaN,NaN,NaN,NaN,715.896079,4.220916e+05
min,101.000000,NaN,NaN,NaN,0.000000,0.000000,23.000000,46.000000,0.000000,38.696300,-9.224400,NaN,NaN,NaN,NaN,2393.000000,8.500000e+04
25%,166.250000,NaN,NaN,NaN,2.000000,1.000000,64.250000,128.500000,0.000000,38.716900,-9.160700,NaN,NaN,NaN,NaN,2881.000000,2.712500e+05
50%,227.500000,NaN,NaN,NaN,2.000000,2.000000,90.000000,180.000000,0.000000,38.730050,-9.136800,NaN,NaN,NaN,NaN,3402.000000,4.500000e+05
75%,288.750000,NaN,NaN,NaN,3.000000,2.000000,145.000000,290.000000,0.000000,38.745800,-9.113575,NaN,NaN,NaN,NaN,3859.000000,7.150000e+05


In [11]:
df_raw['Condition'].value_counts()

New                  96
As New               54
Used                 51
For Refurbishment    45
Name: Condition, dtype: int64

In [13]:
df_raw['PropertyType'].value_counts(
)

Homes                239
Single Habitation      7
Name: PropertyType, dtype: int64

In [14]:
df_raw['PropertySubType'].value_counts()

Apartment             226
Duplex                  9
Townhouse Dwelling      4
Dwelling                2
Studio                  2
Isolated Villa          1
Penthouse               1
Apart Hotel             1
Name: PropertySubType, dtype: int64

In [15]:
df_raw['Country'].value_counts()

Portugal    246
Name: Country, dtype: int64

In [16]:
df_raw['District'].value_counts()

Lisboa    246
Name: District, dtype: int64

In [18]:
df_raw['Municipality'].value_counts()

Lisboa    246
Name: Municipality, dtype: int64

In [19]:
df_raw['Parish'].value_counts()

Marvila                    51
Campo de Ourique           23
Estrela                    17
Santa Maria Maior          17
Arroios                    16
Sao Vicente                14
Penha de Franca            12
Olivais                    11
Belem                      11
Santo Antonio               8
Parque das Nacoes           8
Alvalade                    8
Avenidas Novas              8
Lumiar                      7
Campolide                   6
Sao Domingos de Benfica     6
Benfica                     5
Ajuda                       5
Misericordia                4
Alcantara                   3
Santa Clara                 2
Carnide                     2
Areeiro                     1
Beato                       1
Name: Parish, dtype: int64

ModuleNotFoundError: No module named 'ggplot'